#Boltzmann Machine

##Downloading the dataset

###ML-100K

In [16]:
!wget "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
!unzip ml-100k.zip
!ls

--2025-03-04 10:50:40--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: 'ml-100k.zip'

     0K .......... .......... .......... .......... ..........  1% 95.5K 50s
    50K .......... .......... .......... .......... ..........  2%  195K 37s
   100K .......... .......... .......... .......... ..........  3% 7.51M 24s
   150K .......... .......... .......... .......... ..........  4% 17.6M 18s
   200K .......... .......... .......... .......... ..........  5%  196K 19s
   250K .......... .......... .......... .......... ..........  6% 7.73M 16s
   300K .......... .......... .......... .......... ..........  7%  200K 17s
   350K .......... .......... .......... .......... ..........  8% 4.08M 14s
   400K .......... .........

###ML-1M

In [32]:
!wget "http://files.grouplens.org/datasets/movielens/ml-1m.zip"
!unzip ml-1m.zip
!ls

--2025-03-04 11:06:39--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: 'ml-1m.zip.2'

     0K .......... .......... .......... .......... ..........  0% 94.9K 60s
    50K .......... .......... .......... .......... ..........  1%  191K 45s
   100K .......... .......... .......... .......... ..........  2% 6.78M 30s
   150K .......... .......... .......... .......... ..........  3% 25.8M 22s
   200K .......... .......... .......... .......... ..........  4%  194K 23s
   250K .......... .......... .......... .......... ..........  5% 10.9M 19s
   300K .......... .......... .......... .......... ..........  6%  194K 20s
   350K .......... .......... .......... .......... ..........  6% 13.7M 18s
   400K .......... .......... 

##Importing the libraries

In [14]:
!pip install torch

   ---------------------------------------- 0.0/204.1 MB ? eta -:--:--
   ---------------------------------------- 0.5/204.1 MB 15.9 MB/s eta 0:00:13
   ---------------------------------------- 1.2/204.1 MB 15.5 MB/s eta 0:00:14
   ---------------------------------------- 1.9/204.1 MB 17.2 MB/s eta 0:00:12
   ---------------------------------------- 2.3/204.1 MB 14.6 MB/s eta 0:00:14
    --------------------------------------- 2.8/204.1 MB 13.7 MB/s eta 0:00:15
    --------------------------------------- 3.3/204.1 MB 13.0 MB/s eta 0:00:16
    --------------------------------------- 4.0/204.1 MB 14.2 MB/s eta 0:00:15
    --------------------------------------- 4.9/204.1 MB 15.7 MB/s eta 0:00:13
   - -------------------------------------- 5.6/204.1 MB 15.4 MB/s eta 0:00:13
   - -------------------------------------- 6.2/204.1 MB 15.7 MB/s eta 0:00:13
   - -------------------------------------- 7.0/204.1 MB 16.1 MB/s eta 0:00:13
   - -------------------------------------- 7.6/204.1 MB 15.

In [20]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

## Importing the dataset


In [34]:
# We won't be using this dataset.
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

## Preparing the training set and the test set


In [36]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

## Getting the number of users and movies


In [38]:
nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))

## Converting the data into an array with users in lines and movies in columns


In [40]:
def convert(data):
  new_data = []
  for id_users in range(1, nb_users + 1):
    id_movies = data[:, 1] [data[:, 0] == id_users]
    id_ratings = data[:, 2] [data[:, 0] == id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data
training_set = convert(training_set)
test_set = convert(test_set)

## Converting the data into Torch tensors


In [42]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

## Converting the ratings into binary ratings 1 (Liked) or 0 (Not Liked)


In [44]:
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

## Creating the architecture of the Neural Network


In [46]:
class RBM():
  def __init__(self, nv, nh):
    self.W = torch.randn(nh, nv)
    self.a = torch.randn(1, nh)
    self.b = torch.randn(1, nv)
  def sample_h(self, x):
    wx = torch.mm(x, self.W.t())
    activation = wx + self.a.expand_as(wx)
    p_h_given_v = torch.sigmoid(activation)
    return p_h_given_v, torch.bernoulli(p_h_given_v)
  def sample_v(self, y):
    wy = torch.mm(y, self.W)
    activation = wy + self.b.expand_as(wy)
    p_v_given_h = torch.sigmoid(activation)
    return p_v_given_h, torch.bernoulli(p_v_given_h)
  def train(self, v0, vk, ph0, phk):
    self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
    self.b += torch.sum((v0 - vk), 0)
    self.a += torch.sum((ph0 - phk), 0)
nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)

## Training the RBM


In [48]:
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(0, nb_users - batch_size, batch_size):
    vk = training_set[id_user : id_user + batch_size]
    v0 = training_set[id_user : id_user + batch_size]
    ph0,_ = rbm.sample_h(v0)
    for k in range(10):
      _,hk = rbm.sample_h(vk)
      _,vk = rbm.sample_v(hk)
      vk[v0<0] = v0[v0<0]
    phk,_ = rbm.sample_h(vk)
    rbm.train(v0, vk, ph0, phk)
    train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))
    s += 1.
  print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))


epoch: 1 loss: tensor(0.3648)
epoch: 2 loss: tensor(0.2527)
epoch: 3 loss: tensor(0.2506)
epoch: 4 loss: tensor(0.2517)
epoch: 5 loss: tensor(0.2478)
epoch: 6 loss: tensor(0.2492)
epoch: 7 loss: tensor(0.2500)
epoch: 8 loss: tensor(0.2461)
epoch: 9 loss: tensor(0.2500)
epoch: 10 loss: tensor(0.2459)


## Testing the RBM


In [50]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.2637)
